-----

### **Course: Advanced Prompt Engineering for Banking**

### **Lab 5: Core Prompting Tasks in Financial Services (Part 1)**

**Objective:** This hands-on lab explores a diverse set of real-world prompting tasks essential for a large financial institution. You will use targeted prompts to handle classification, code generation, and information extraction. By comparing naive prompts with improved, structured prompts, you will learn to generate more reliable and useful outputs for technical applications.

**Instructions:** Run the code for both the "Naive Prompt" and the "Improved Prompt" in each task to observe the significant difference in the quality and structure of the LLM's response.

-----

### **Consolidated Use Case List (13 Tasks)**

Here is the complete list of the 13 practical tasks we will cover in this notebook:

1.  **Zero-Shot Classification:** Classifying sentiment of financial news headlines.
2.  **SQL Code Generation:** Querying a client/deals database from a schema.
3.  **Information Extraction:** Parsing client emails into structured JSON.
4.  **Closed-Domain Question Answering:** Extracting facts from a market research report.
5.  **Document Summarization:** Creating constrained executive briefings from compliance alerts.
6.  **Transformation:** Creating annotated training data for an NER model.
7.  **Translation:** Translating banking alerts into multiple languages (JSON output).
8.  **Code Explanation and Refactoring:** Analyzing and improving legacy Python code.
9.  **Synthetic Data Generation:** Creating realistic CSV transaction data for testing.
10. **Root Cause Analysis:** Diagnosing system outages from monitoring alerts.
11. **Portfolio Analysis (CoT):** Calculating asset allocation, concentration risk, and Sharpe Ratio.
12. **Complex Macroeconomic Analysis:** Extracting key signals from central bank statements into JSON.
13. **Financial Markets Strategy Generation (ToT):** Devising and evaluating multiple trading strategies based on a live market event.

### **Setup**

This lab uses the `langchain` library. The following setup will initialize the LLM and a helper function to execute our prompts.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import Markdown, display

# Load environment variables
load_dotenv()

# Initialize the model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Helper function to execute prompts and print the prompt text
def execute_prompt(prompt_template, title, **kwargs):
    """
    Executes a given prompt template and prints the prompt and response.
    """
    print(f"--- {title} ---")

    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()

    print("PROMPT:")
    print(prompt.format(**kwargs))

    response = chain.invoke(kwargs)

    print("\n" + "="*20 + " RESPONSE " + "="*20 + "\n")
    display(Markdown(response))
    print("\n" + "-"*50 + "\n")

ModuleNotFoundError: No module named 'langchain_openai'

-----

### **Task 1: Zero-Shot Classification**

**Concept:** This technique involves asking the model to classify text into predefined categories without providing any prior examples in the prompt. It relies on the model's vast pre-existing knowledge to understand the context and make an accurate classification.

**Scenario:** An automated system for a large investment bank scans news feeds for articles relevant to its portfolio. The system needs to classify the sentiment of each news headline to gauge market mood for an algorithmic trading dashboard. The output must be a single, standardized word for easy processing.

In [ ]:
news_headline = "Global markets surge as the Federal Reserve announces an unexpected interest rate cut to boost the economy."

#### **Naive Prompt 👎**

This prompt is too conversational. It doesn't specify the desired output format, leading to a verbose and non-standardized response that is difficult for a downstream application to parse.

**Code:**

In [ ]:
naive_prompt_1 = "What do you think the sentiment of this news headline is?\n\nHeadline: \"{headline}\""

execute_prompt(
    naive_prompt_1,
    "1.1 Naive Sentiment Classification",
    headline=news_headline
)

#### **Improved Zero-Shot Prompt 👍**

This prompt is precise. It assigns a role, clearly defines the task, and provides strict constraints on the output format, ensuring a clean, predictable, and machine-readable response.

**Code:**

In [ ]:
improved_prompt_1 = """
Act as a financial market analyst AI.
Given the following news headline, classify the market sentiment.

The sentiment must be one of the following: Positive, Negative, or Neutral.
Return only the single-word sentiment category and nothing else.

Headline: "{headline}"
"""
execute_prompt(
    improved_prompt_1,
    "1.2 Improved Zero-Shot Classification",
    headline=news_headline
)

-----

### **Task 2: SQL Code Generation**

**Concept:** LLMs can act as powerful co-pilots for technical tasks. By providing a database schema and a clear, natural language request, you can generate accurate SQL queries, which is a common task for developers and data engineers in a financial firm.

**Scenario:** A developer needs to query the firm's client database to find the names of investment banking clients who have completed a deal with a value greater than $1 billion in the "Technology" sector.

In [ ]:
db_schema = """
Table: 'Clients', Columns: [ClientID, ClientName, Sector]
Table: 'Deals', Columns: [DealID, ClientID, DealValueUSD, DealDate]
"""
request = "Find the names of all clients in the 'Technology' sector who have a deal worth more than $1,000,000,000."

#### **Naive Prompt 👎**

This prompt is dangerous because it lacks the necessary context (the database schema). The LLM is forced to guess the table and column names, which will almost certainly lead to a non-functional or incorrect SQL query.

**Code:**

In [ ]:
naive_prompt_2 = "Write a SQL query for our database to get the names of tech clients with deals over a billion dollars."

execute_prompt(
    naive_prompt_2,
    "2.1 Naive SQL Generation"
)

#### **Improved Prompt 👍**

This prompt provides all the necessary context. It clearly defines the database schema, allowing the model to ground its response in the actual structure of the data and generate a correct, executable query.

**Code:**

In [ ]:
improved_prompt_2 = """
Act as an expert SQL developer for a large investment bank.
Carefully review the following database schema.

Schema:
{schema}

Your task is to generate a single, accurate MySQL query to fulfill the following request.

Request: "{sql_request}"
"""
execute_prompt(
    improved_prompt_2,
    "2.2 Improved SQL Generation",
    schema=db_schema,
    sql_request=request
)

-----

### **Task 3: Information Extraction**

**Concept:** This task involves identifying and pulling out specific pieces of information (entities) from unstructured text and organizing them into a structured format like JSON. This is crucial for developers building data ingestion pipelines.

**Scenario:** A software engineer is building a tool for the wealth management division. The tool needs to parse an email from a prospective client and extract key details to automatically create a profile in the Client Relationship Management (CRM) system via an API call that expects a specific JSON format.

In [ ]:
client_email = """
Hi, my name is a new prospect. I'm based in a major financial hub and I am interested in your private wealth services. I'm a director at a major tech firm, with an annual income of about ₹5 Crore. My current investment portfolio is around ₹20 Crore, and I would describe my risk tolerance as 'growth-oriented'. You can reach me at prospect.new@email.com.
"""

#### **Naive Prompt 👎**

This prompt is too vague. It asks to "extract details" but doesn't specify the desired output format. The result will be an unstructured paragraph, which cannot be used in an automated, programmatic workflow.

**Code:**

In [ ]:
naive_prompt_3 = "Please extract the key details from this client email.\n\nEmail:\n{email_text}"

execute_prompt(
    naive_prompt_3,
    "3.1 Naive Information Extraction",
    email_text=client_email
)

#### **Improved Prompt 👍**

This prompt is built for automation. It instructs the model to produce a JSON object and specifies the exact keys and schema required by the CRM's API. This ensures the output is always consistent and machine-readable.

**Code:**

In [ ]:
improved_prompt_3 = """
Act as a highly accurate data extraction AI for a wealth management division's CRM system.
Extract the key information from the client email below.

The output must be a valid JSON object with the following keys: "name", "location", "email", "annual_income_inr", "portfolio_value_inr", "risk_profile".
If a value is not found, use "N/A". Do not add any text before or after the JSON object.

Email:
{email_text}
"""
execute_prompt(
    improved_prompt_3,
    "3.2 Improved Information Extraction",
    email_text=client_email
)


-----

### **Task 4: Closed-Domain Question Answering**

**Concept:** In Closed-Domain QA, the model is provided with a specific piece of context and asked a question whose answer is contained within that text. This is a fundamental technique for developers building Retrieval-Augmented Generation (RAG) systems, as it forces the model to rely on provided facts rather than its general knowledge.

**Scenario:** A developer is building a feature for an internal research portal. The feature must allow analysts to ask questions about a financial report and receive answers based *only* on the report's text.

In [ ]:
market_report_excerpt = """
In the second quarter of 2025, the global mergers and acquisitions (M&A) market saw a significant downturn, with total deal volume decreasing by 22% compared to the same period last year. The technology sector, while still the most active, experienced a 30% drop in deal value. In contrast, the renewable energy sector bucked the trend, showing a 15% increase in M&A activity, driven by new government incentives and a push towards sustainable investing. The total value of all M&A deals recorded in Q2 2025 was $750 billion.
"""

question = "By what percentage did the renewable energy sector's M&A activity change?"

#### **Naive Prompt 👎**

This prompt is dangerous for factual queries. By not providing the context, it forces the model to search its general knowledge, which might be outdated, incorrect, or from an unrelated report, leading to a confident but wrong answer (hallucination).

**Code:**

In [ ]:
naive_prompt_4 = "{user_question}"

execute_prompt(
    naive_prompt_4,
    "4.1 Naive (Open-Domain) Q&A",
    user_question=question
)

#### **Improved Prompt 👍**

This prompt provides the specific `Context` and explicitly instructs the model to answer *only* from that text. This creates a reliable, closed-domain system where the source of the answer is known and verifiable.

**Code:**

In [ ]:
improved_prompt_4 = """
You are a research assistant AI.
Using only the context from the market report provided below, answer the question accurately. If the answer is not in the text, state "Information not available in the provided context."

Context:
{report}

Question:
{user_question}
"""
execute_prompt(
    improved_prompt_4,
    "4.2 Improved (Closed-Domain) Q&A",
    report=market_report_excerpt,
    user_question=question
)

-----

### **Task 5: Document Summarization**

**Concept:** Summarization involves creating a short, concise version of a long document while retaining the most critical information. For technical applications, it's crucial to control the length and format of the summary.

**Scenario:** A development team is building a tool for the compliance department. The tool ingests lengthy internal policy update alerts and generates a brief, structured summary for a daily executive briefing dashboard.

In [ ]:
compliance_alert = """
Subject: Urgent Policy Update - Cross-Border Data Residency Requirements
Effective immediately, all client-identifying data associated with European Union customers must be stored and processed exclusively on servers located within EU member states, in accordance with the new GDPR-Plus directive. This policy supersedes all previous guidance. Application teams must conduct an immediate audit of their data storage protocols to identify non-compliant data instances. All data migration projects to bring EU data into compliance must be completed by the end of the fiscal quarter. Failure to comply will result in significant regulatory penalties and potential suspension of services. Please review the attached 50-page technical implementation guide for full details.
"""

#### **Naive Prompt 👎**

A simple "summarize this" command is unreliable for an automated pipeline. The output length, style, and format will be unpredictable, making it difficult to display consistently in a UI or dashboard.

**Code:**

In [ ]:
naive_prompt_5 = "Summarize this compliance alert:\n\n{alert}"

execute_prompt(
    naive_prompt_5,
    "5.1 Naive Summarization",
    alert=compliance_alert
)

#### **Improved Prompt 👍**

This prompt applies specific constraints to the output. It dictates the exact format (bullet points), the number of points, and the style (complete sentences), ensuring the summary is always concise, structured, and suitable for the executive dashboard.

**Code:**

In [ ]:
improved_prompt_5 = """
Act as a compliance summary AI for a financial institution.
Summarize the following compliance alert in exactly three concise bullet points. Each bullet point must be a single, complete sentence.

Alert:
{alert}
"""
execute_prompt(
    improved_prompt_5,
    "5.2 Constrained Summarization",
    alert=compliance_alert
)

-----

### **Task 6: Transformation**

**Concept:** Transformation involves converting content from one format or structure to another. This is a common task for developers, especially when preparing data for machine learning models or integrating different systems.

**Scenario:** A developer on a machine learning team needs to create annotated training data for a custom Named Entity Recognition (NER) model. They have raw sentences from financial news and need to transform them into an XML-like format that their training script can parse.

In [ ]:
news_sentence = "First National Bank announced a merger with Sterling Financial Group for $5 billion."

#### **Naive Prompt 👎**

This prompt correctly identifies the task but doesn't specify the required output format. The model will likely return a simple list, which is helpful for a human but useless for the automated training script that expects a specific annotation schema.

**Code:**

In [ ]:
naive_prompt_6 = "Identify the organizations and money values in this sentence: \"{sentence}\""

execute_prompt(
    naive_prompt_6,
    "6.1 Naive Entity Recognition",
    sentence=news_sentence
)

#### **Improved Prompt 👍**

This prompt clearly defines the transformation rule. It tells the model exactly how to format the output by providing the specific tags (`<ORG>`, `<MONEY>`) to use. This creates perfectly formatted training data programmatically.

**Code:**

In [ ]:
improved_prompt_6 = """
You are a data annotation tool for an ML model. Your task is to transform the given sentence by wrapping specific entities in XML tags.
- Wrap all organization names in `<ORG>` tags.
- Wrap all monetary values in `<MONEY>` tags.

Do not identify any other entities.

Sentence: "{sentence}"
"""
execute_prompt(
    improved_prompt_6,
    "6.2 Structured Data Transformation",
    sentence=news_sentence
)

-----

### **Task 7: Translation**

**Concept:** LLMs have strong multilingual capabilities. This task involves translating text between languages, a critical function for global financial firms.

**Scenario:** A developer is internationalizing a mobile banking application. They need to translate standard system alerts into multiple languages while maintaining a professional tone and ensuring the output is in a structured format (JSON) for easy integration into the app's language files.

In [ ]:
alert_text_english = "Your wire transfer of $10,000.00 to account ending in 4567 has been successfully processed."

#### **Naive Prompt 👎**

This prompt is too simple. It will produce a plain text translation, but the tone might be too informal. More importantly, it doesn't return the data in a structured way, forcing the developer to manually copy-paste and format the results.

**Code:**

In [ ]:
naive_prompt_7 = "Translate this to French and Japanese: \"{alert}\""

execute_prompt(
    naive_prompt_7,
    "7.1 Naive Translation",
    alert=alert_text_english
)

#### **Improved Prompt 👍**

This prompt is designed for a developer's workflow. It assigns a professional persona, specifies the exact target languages, and—most critically—demands the output in a clean JSON format. This response can be parsed directly and integrated into the application's codebase without manual intervention.

**Code:**

In [ ]:
improved_prompt_7 = """
You are a professional translator for a global financial institution.
Translate the following banking alert into French and Japanese. Maintain a formal and professional tone appropriate for a banking context.

Return the output as a single, valid JSON object with keys "fr" for French and "ja" for Japanese.

English Alert: "{alert}"
"""
execute_prompt(
    improved_prompt_7,
    "7.2 Structured Translation",
    alert=alert_text_english
)


-----

### **Task 8: Code Explanation and Refactoring**

**Concept:** LLMs can act as powerful programming assistants. For developers, a key use case is analyzing legacy or poorly documented code to understand its functionality, identify potential issues, and refactor it for better clarity and maintainability.

**Scenario:** A junior developer on a quantitative analysis team is tasked with debugging a legacy Python function. The function is critical for a risk model, but it is uncommented and uses non-descriptive variable names, making it hard to understand.

In [ ]:
legacy_code = """
def calc(d, i, s):
    w1 = 0.5
    w2 = 0.3
    w3 = 0.2
    if s > 750:
        s_factor = 0.1
    else:
        s_factor = 1.0
    val = (d / i * w1) + (s_factor * w2) + (1/s * w3)
    return val
"""

#### **Naive Prompt 👎**

This prompt will provide a basic explanation, but it's not a comprehensive analysis. It doesn't help the developer improve the code or spot potential flaws, which is the ultimate goal.

**Code:**

`

In [ ]:
naive_prompt_8 = "Explain what this Python function does:\n\n```python\n{code}\n```"

execute_prompt(
    naive_prompt_8,
    "8.1 Naive Code Explanation",
    code=legacy_code
)


#### **Improved Prompt 👍**

This prompt is structured as a multi-step task for a senior developer persona. It asks not only for an explanation but also for a practical, refactored version of the code and a critical analysis of its potential issues, providing far more value to the developer.


In [ ]:
improved_prompt_8 = """
You are a senior Python developer conducting a code review. Your task is to analyze and refactor the following legacy function.

Legacy Function:
{code}


Please provide the following:

1.  **Explanation:** In plain English, describe the function's purpose and the weighted calculation it performs.
2.  **Refactored Code:** Rewrite the function to use descriptive variable names and add comments explaining the logic.
3.  **Potential Issue:** Identify one potential runtime error or logical flaw in the original code.
"""

execute_prompt(
improved_prompt_8,
"8.2 Improved Code Analysis and Refactoring",
code=legacy_code
)



---
### **Task 9: Synthetic Data Generation**

**Concept:** Developers frequently need realistic data for testing applications, running performance benchmarks, or training machine learning models. LLMs can generate high-quality synthetic (fake) data that mimics the structure and format of real data without violating privacy or using sensitive information.

**Scenario:** A developer is building a new data ingestion service for a fraud detection system. To test the service's parsing and validation logic, they need a small but diverse set of sample transaction data in a specific CSV format.

#### **Naive Prompt 👎**
This prompt is too vague. It will generate some data, but the format will likely be inconsistent, the values may not be realistic, and it might not include the variety needed to properly test the system (e.g., different currencies or transaction types).


In [ ]:
naive_prompt_9 = "Generate some sample banking transaction data for testing."

execute_prompt(
    naive_prompt_9,
    "9.1 Naive Data Generation"
)

#### **Improved Prompt 👍**

This prompt acts as a detailed specification for the data generator. It defines the exact CSV header, specifies the data types and formats for each column, and requests specific variations in the data, ensuring the output is perfectly suited for the developer's testing needs.

In [ ]:
improved_prompt_9 = """
You are a test data generation utility. Your task is to create a list of 10 synthetic banking transactions in a raw CSV format.

**Requirements:**
1.  The output must start with the following CSV header row: `TransactionID,Timestamp,Amount,Currency,MerchantName,TransactionType`
2.  `TransactionID` must be a unique UUID.
3.  `Timestamp` must be in ISO 8601 format (e.g., YYYY-MM-DDTHH:MM:SSZ).
4.  The data must include at least two different currencies (e.g., 'USD', 'EUR').
5.  The data must include a mix of `TransactionType` such as 'Purchase', 'Refund', and 'Subscription'.
6.  Do not add any introductory text, explanations, or code block formatting. Return only the raw CSV data.
"""
execute_prompt(
    improved_prompt_9,
    "9.2 Structured Synthetic Data Generation"
)

-----

### **Task 10: Root Cause Analysis**

**Concept:** LLMs can perform reasoning over a sequence of events. For technical teams, this can be applied to incident management by analyzing a series of monitoring alerts to suggest a likely root cause, helping engineers diagnose problems faster.

**Scenario:** A developer on the Site Reliability Engineering (SRE) team is investigating a critical outage of the client portfolio management system. They have a chronological list of alerts from the monitoring system and need to quickly form a hypothesis about what happened.

In [ ]:
alert_sequence = """
- 02:05:01 UTC: Database CPU utilization exceeds 95% (Alert)
- 02:05:15 UTC: Query execution time for 'client_portfolio' table exceeds 30 seconds (Alert)
- 02:06:00 UTC: Application server 'app-prod-01' reports connection pool exhaustion (Critical)
- 02:06:30 UTC: API Gateway reports 503 Service Unavailable for /v1/portfolio endpoint (Critical)
- 02:07:00 UTC: End-user reports of 'Portfolio loading failed' begin flooding support channels.
"""

#### **Naive Prompt 👎**

This prompt will simply re-state the events in prose. It provides a summary but doesn't perform the higher-level reasoning needed to identify a root cause or suggest an action, which is what the engineer truly needs.

**Code:**

In [ ]:
naive_prompt_10 = "What happened here based on these alerts?\n\n{alerts}"

execute_prompt(
    naive_prompt_10,
    "10.1 Naive Incident Summary",
    alerts=alert_sequence
)

#### **Improved Prompt 👍**

This prompt instructs the model to act as a specific expert (an SRE) and perform a structured analysis. It asks the model to differentiate between the symptom and the cause and to propose a concrete next step, turning a simple list of events into an actionable incident hypothesis.

**Code:**

In [ ]:
improved_prompt_10 = """
You are an expert Site Reliability Engineer (SRE). Analyze the following sequence of monitoring alerts to determine the likely root cause of the incident.

Alert Sequence:
{alerts}

Provide your analysis in the following structure:
1.  **User-Facing Symptom:** What was the primary impact experienced by end-users?
2.  **Likely Root Cause:** Based on the timeline, what is the most probable underlying technical issue that triggered the entire event chain?
3.  **Suggested Immediate Action:** What is the first concrete step the on-call engineer should take to begin mitigation?
"""
execute_prompt(
    improved_prompt_10,
    "10.2 Structured Root Cause Analysis",
    alerts=alert_sequence
)


-----

### **Task 11: Portfolio Analysis**

**Concept:** This task uses a multi-step reasoning approach (Chain-of-Thought) to analyze a client's investment portfolio. It involves not just understanding the data but also performing calculations and deriving insights, which is a core function for developers building robo-advisory or portfolio management tools.

**Scenario:** A developer is building a service that automatically generates a high-level analysis of a client's portfolio. The service needs to identify concentration risk and calculate the Sharpe Ratio, a key measure of risk-adjusted return.

In [ ]:
portfolio_data = """
{
  "assets": [
    {"ticker": "TECH_ETF", "value": 60000, "asset_class": "Equity", "region": "US"},
    {"ticker": "HEALTH_ETF", "value": 20000, "asset_class": "Equity", "region": "US"},
    {"ticker": "BOND_FUND", "value": 20000, "asset_class": "Fixed Income", "region": "Global"}
  ],
  "portfolio_return": 0.12,
  "portfolio_std_dev": 0.18,
  "risk_free_rate": 0.04
}
"""

#### **Naive Prompt 👎**

This prompt is too vague. It will produce a generic, unstructured summary of the portfolio's contents without performing the specific, quantitative analysis required for a financial application.

**Code:**

In [ ]:
naive_prompt_11 = "Analyze this investment portfolio and tell me about it:\n\n{portfolio}"

execute_prompt(
    naive_prompt_11,
    "11.1 Naive Portfolio Analysis",
    portfolio=portfolio_data
)

#### **Improved CoT Prompt 👍**

This prompt instructs the model to act as a quantitative analyst and follow a clear, step-by-step reasoning process. It forces the model to perform specific calculations and structure its output, making the analysis transparent, auditable, and useful.

**Code:**

In [ ]:
improved_prompt_11 = """
You are a quantitative analyst AI. Analyze the provided investment portfolio data. Let's think step-by-step to produce a structured analysis.

Portfolio Data:
{portfolio}

**Analysis Steps:**
1.  **Calculate Asset Allocation:** Determine the percentage allocation for each asset class (Equity, Fixed Income).
2.  **Identify Concentration Risk:** Based on the allocation, state the primary concentration risk (e.g., asset class, region).
3.  **Calculate Sharpe Ratio:** Calculate the Sharpe Ratio using the formula: (Portfolio Return - Risk-Free Rate) / Portfolio Standard Deviation. Show your calculation.
4.  **Summarize Findings:** Provide a concise, two-sentence summary of the portfolio's key characteristics based on the steps above.
"""
execute_prompt(
    improved_prompt_11,
    "11.2 Improved Portfolio Analysis (CoT)",
    portfolio=portfolio_data
)

-----

### **Task 12: Complex Macroeconomic Analysis**

**Concept:** This task involves parsing dense, jargon-filled text from sources like central bank statements to extract nuanced, forward-looking insights. This is a critical capability for developers building tools for economics research or algorithmic trading teams.

**Scenario:** A developer is creating a tool that ingests press releases from central bank monetary policy meetings. The tool must extract the key outlooks and policy signals into a structured format for the firm's economists.

In [ ]:
central_bank_statement = """
The Monetary Policy Committee noted that while headline inflation has moderated to 4.8% due to favorable base effects, core inflation remains persistent, driven by robust domestic demand and elevated service sector costs. GDP growth forecasts for the fiscal year are revised upwards from 6.5% to 6.8%, reflecting strong manufacturing output. However, the committee maintains a hawkish stance, signaling that a 'higher for longer' interest rate environment may be necessary to anchor inflationary expectations. The committee will remain data-dependent in its upcoming decisions.
"""

#### **Naive Prompt 👎**

A generic summary request will capture the main ideas but is likely to miss the subtle but critical signals that economists look for, such as the specific policy stance or forward guidance. The unstructured output is also difficult to use programmatically.

**Code:**

In [ ]:
naive_prompt_12 = "Summarize this central bank statement:\n\n{statement}"

execute_prompt(
    naive_prompt_12,
    "12.1 Naive Macroeconomic Summary",
    statement=central_bank_statement
)

#### **Improved Prompt 👍**

This prompt is designed for a technical audience. It instructs the model to act as a specialist assistant and extract specific, predefined data points into a clean JSON object. This structured output can be directly ingested by other systems for analysis or dashboarding.

**Code:**

In [ ]:
improved_prompt_12 = """
You are an AI assistant for a macroeconomist. Analyze the following central bank policy statement and extract specific insights.

Statement:
{statement}

Return a single JSON object with the following keys:
- "inflation_outlook": A brief summary of the inflation situation.
- "gdp_growth_outlook": A brief summary of the economic growth forecast.
- "policy_stance": Classify as "Hawkish", "Dovish", or "Neutral".
- "forward_guidance": A brief summary of the hint about future policy actions.
"""
execute_prompt(
    improved_prompt_12,
    "12.2 Structured Macroeconomic Analysis",
    statement=central_bank_statement
)

-----

### **Task 13: Financial Markets Strategy Generation**

**Concept:** This task uses a "Tree-of-Thought" style approach, where the model is prompted to explore and evaluate multiple alternative strategies in response to a complex event. This is valuable for developers creating decision-support tools for traders and portfolio managers.

**Scenario:** A developer is building a real-time decision-support tool for the equities trading desk. When a major market event occurs, the tool should propose and evaluate several potential trading strategies.

In [ ]:
market_event = """
Market Event: The U.S. Consumer Price Index (CPI) report was just released. Headline inflation is 20 basis points higher than consensus estimates. This unexpected result is causing high volatility in equity futures and the US dollar.
"""

#### **Naive Prompt 👎**

This prompt asks for a single course of action. It will likely generate a generic, one-dimensional piece of advice that lacks the nuance, risk assessment, and consideration of alternatives that a professional trader requires.

**Code:**

In [ ]:
naive_prompt_13 = "What is a good trading strategy based on this CPI report?\n\n{event}"

execute_prompt(
    naive_prompt_13,
    "13.1 Naive Trading Advice",
    event=market_event
)

#### **Improved ToT-style Prompt 👍**

This prompt forces the model to think like a strategist. It must generate multiple, distinct strategies, and for each one, it must provide a rationale (the "why") and the associated risks (the "what if"). This structured evaluation of alternatives provides a much richer, more valuable output for decision support.

**Code:**

In [ ]:
improved_prompt_13 = """
You are an AI trading strategist. Analyze the provided real-time market event. Your task is to devise and evaluate potential trading strategies.

Market Event:
{event}

Please structure your response as follows:
1.  **Strategy 1 (Short-Term):** Propose a short-term trading strategy.
    -   **Rationale:** Explain the logic behind this trade.
    -   **Primary Risk:** State the main risk that could invalidate this strategy.
2.  **Strategy 2 (Longer-Term):** Propose a longer-term portfolio adjustment strategy.
    -   **Rationale:** Explain the logic behind this adjustment.
    -   **Primary Risk:** State the main risk associated with this view.
"""
execute_prompt(
    improved_prompt_13,
    "13.2 ToT-style Strategy Generation",
    event=market_event
)